In [1]:
import glob, os
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

In [2]:
print(os.getcwd())

/Users/psrw2/OneDrive - University Of Cambridge/GitHub/CAMDA-DILI/Machine_Learning/code


In [3]:
os.chdir('../data/FDA_Validation_Set_Results')

In [4]:
directories = ['./Predictions_ECFP4','./Predictions_MD','.Predictions_PT']

In [5]:
def metrics_func(TP,FP,FN,TN):

    
    if TN == 0:
        specificity=0
        recall = TP/(TP+FN)
        precision = TP/(TP+FP)
        if (TN+FN) == 0:
            MCC = 0
        else:
            MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))

        
    elif TP == 0:
        precision = 0
        recall=0
        specificity = TN/(TN+FP)
        if (FP+TP) == 0:
            MCC = 0
        else:
            MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))


        
#    elif (TN+FN) ==0 or (FP+TP) == 0:
#        MCC=0
        
    else:
        specificity = TN/(TN+FP)
        recall = TP/(TP+FN)
        MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))
        precision = TP/(TP+FP)
    
    
    balanced_accuracy = (specificity+recall)/2
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    
    return precision,recall,specificity,balanced_accuracy,accuracy,MCC

In [6]:
faulty_file_dirs = []
for result_dir in directories:
    #df = pd.DataFrame(data=None, columns=['model','precision','recall','balanced_accuracy','MCC'])
    list_of_lists = []
    for file in os.listdir(result_dir):
        if file.endswith(".txt"):
            print(os.path.join(result_dir, file))
            f = open(os.path.join(result_dir, file), 'r')
            try:

                content = [line.rstrip('\n') for line in f]
                print(content)
                
                
                size=sum(1 for _ in content)
                print(size)
                
                
                print(type(content[1].split(",")[0]),content[1].split(",")[0])
                
                p = content[1].split(",")[0].replace('"', '')
                print(p)
                if p == str("1"): # if never predicted 0 missing line
                    print("wow1")
                    TN = 0
                    FN = 0
                    FP = int(content[1].split(",")[1])
                    TP = int(content[1].split(",")[2])
                    
                elif p == str("0") and size == 2 : # if never predicted 1
                    print("wow2")
                    TN = int(content[1].split(",")[1])
                    FN = int(content[1].split(",")[2])
                    FP = 0
                    TP = 0
                    print(TN,FN,FP,TP)

                else:

                    TN = int(content[1].split(",")[1])
                    FN = int(content[1].split(",")[2])
                    FP = int(content[2].split(",")[1])
                    TP = int(content[2].split(",")[2])


                p,r,sen,ba,a,mcc = metrics_func(TP,FP,FN,TN)

                list_of_lists.append([file,p,r,sen,ba,a,mcc])
                
            except:
                print('ERROR')
                break
                
                faulty_file_dirs.append(os.path.join(result_dir, file))
                list_of_lists.append([file,None,None,None,None,None,None])
                
            finally:
                f.close()
    df = pd.DataFrame(list_of_lists,columns=['model','precision','recall','specificity','balanced_accuracy','accuracy','MCC'])
    df['model'] = df['model'].str[4:]
    df['model'] = df['model'].str[:-4]
    df.replace('_', '.', regex=True, inplace=True)
    #df['model'] = df['model'].replace("_", ".")
    df.rename(columns={"model": "splits"},inplace=True)
    df.to_csv(result_dir+"/ambiguous_results.csv")

./Predictions_ECFP4/val.MCLCNC_SVM_1_7.txt
['"0","1"', '"0",13,5', '"1",19,12']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.all_RF_1_1.txt
['"0","1"', '"0",16,4', '"1",16,13']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.all_SVM_1_10.txt
['"0","1"', '"0",14,6', '"1",18,11']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.MCLCNC_SVM_1_6.txt
['"0","1"', '"0",13,5', '"1",19,12']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.MCLCNC_SVM_1_4.txt
['"0","1"', '"0",13,5', '"1",19,12']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.all_RF_1_2.txt
['"0","1"', '"0",15,3', '"1",17,14']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.all_RF_1_3.txt
['"0","1"', '"0",14,4', '"1",18,13']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.MCLCNC_SVM_1_5.txt
['"0","1"', '"0",13,5', '"1",19,12']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.MCLCNC_SVM_1_1.txt
['"0","1"', '"0",13,5', '"1",19,12']
3
<class 'str'> "0"
0
./Predictions_ECFP4/val.all_RF_1_7.txt
['"0","1"', '"0",15,3', '"1",17,14']
3
<class 'str'> "0"


FileNotFoundError: [Errno 2] No such file or directory: '.Predictions_PT'

In [7]:
faulty_file_dirs

[]

In [8]:
df

,splits,precision,recall,specificity,balanced_accuracy,accuracy,MCC
0,MCLCNC.SVM.1.7,0.277778,0.588235,0.18750,0.387868,0.326531,-0.241790
1,all.RF.1.1,0.347826,0.470588,0.53125,0.500919,0.510204,0.001753
2,all.SVM.1.10,0.346154,0.529412,0.46875,0.499081,0.489796,-0.001753
3,MCLCNC.SVM.1.6,0.333333,0.647059,0.31250,0.479779,0.428571,-0.041049
4,MCLCNC.SVM.1.4,0.300000,0.529412,0.34375,0.436581,0.408163,-0.123912
5,all.RF.1.2,0.370370,0.588235,0.46875,0.528493,0.510204,0.054534
6,all.RF.1.3,0.428571,0.705882,0.50000,0.602941,0.571429,0.198030
7,MCLCNC.SVM.1.5,0.314286,0.647059,0.25000,0.448529,0.387755,-0.108465
8,MCLCNC.SVM.1.1,0.343750,0.647059,0.34375,0.495404,0.448980,-0.009191
9,all.RF.1.7,0.400000,0.588235,0.53125,0.559743,0.551020,0.113773
